<a href="https://colab.research.google.com/github/Aleesia/2020-knu-df/blob/a.melekestseva/Compete!_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip -q  "/content/gdrive/My Drive/deep-fake-detection-knu-2020.zip"

In [0]:
import os
fake_images = os.listdir('train/train/1_fake')
train_fakes = fake_images[:51000]
val_fakes = fake_images[51000:]
real_images = os.listdir('train/train/0_real')
train_reals = real_images[:51000]
val_reals = real_images[51000:]

def copyfile(filename, fromdir, todir):
    fromfullpath = os.path.join(fromdir, filename)
    tofullpath = os.path.join(todir, filename)
    fromfile = open(fromfullpath, "rb")
    tofile = open(tofullpath, "wb")
    cnt = fromfile.read()
    tofile.write(cnt)
    fromfile.close()
    tofile.close()

os.mkdir('my_data')
os.mkdir('my_data/train')
os.mkdir('my_data/val')
os.mkdir('my_data/train/1_fake')
os.mkdir('my_data/val/1_fake')
os.mkdir('my_data/train/0_real')
os.mkdir('my_data/val/0_real')

for image in train_fakes:
    copyfile(image, 'train/train/1_fake', 'my_data/train/1_fake')
    os.remove('train/train/1_fake/'+image)
for image in val_fakes:
    copyfile(image, 'train/train/1_fake', 'my_data/val/1_fake')
    os.remove('train/train/1_fake/'+image)
for image in train_reals:
    copyfile(image, 'train/train/0_real', 'my_data/train/0_real')
    os.remove('train/train/0_real/'+image)
for image in val_reals:
    copyfile(image, 'train/train/0_real', 'my_data/val/0_real')
    os.remove('train/train/0_real/'+image)

In [5]:
print(len(os.listdir('my_data/val/1_fake')))
print(len(os.listdir('my_data/val/0_real')))
print(len(os.listdir('my_data/train/1_fake')))
print(len(os.listdir('my_data/train/0_real')))

9000
9000
51000
51000


In [6]:
BATCH_SIZE = 64
train_data_path = 'my_data/train/'
val_data_path = 'my_data/val/'
VAL_SIZE = 0.15 # percentage of data for validation

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
         transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
val_dataset = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE,
        shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = BATCH_SIZE, num_workers=2)

print(f"Length train: {len(train_loader.dataset)}")
print(f"Length valid: {len(val_loader.dataset)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Length train: 102000
Length valid: 18000


In [0]:
test_data_path = "/content/test"
test_dataset = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE,
        shuffle=False, num_workers=2)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [0]:
model = Net().to(device)
import math


def log_loss(output, target):
    if output==1:
        return 0
    return target * math.log(output) + (1-target) * math.log(1-output)

criterion = nn.CrossEntropyLoss()
lr = 0.05
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [40]:
epoch_num = 12
for epoch in range(1, epoch_num+1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[Epoch {epoch}: batch {i + 1}] loss: {running_loss / 200}')
            running_loss = 0.0
    if epoch % 4 == 0:
        lr = lr*0.7
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fb9f84c4fd0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fb9f84c4fd0>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fb9fc4587b8>>
Traceback (most rece

[Epoch 1: batch 200] loss: 0.6933954557776452
[Epoch 1: batch 400] loss: 0.6893755969405174
[Epoch 1: batch 600] loss: 0.6702169981598854
[Epoch 1: batch 800] loss: 0.6513182288408279
[Epoch 1: batch 1000] loss: 0.6249903136491776
[Epoch 1: batch 1200] loss: 0.5925528402626514
[Epoch 1: batch 1400] loss: 0.5731888145208359
[Epoch 2: batch 200] loss: 0.5460864414274692
[Epoch 2: batch 400] loss: 0.5425091649591923
[Epoch 2: batch 600] loss: 0.5352892991900444
[Epoch 2: batch 800] loss: 0.5219214032590389
[Epoch 2: batch 1000] loss: 0.5100987572968007
[Epoch 2: batch 1200] loss: 0.506058894842863
[Epoch 2: batch 1400] loss: 0.506723916977644
[Epoch 3: batch 200] loss: 0.4969533234834671
[Epoch 3: batch 400] loss: 0.48572642982006076
[Epoch 3: batch 600] loss: 0.4978052355349064
[Epoch 3: batch 800] loss: 0.48603402704000476
[Epoch 3: batch 1000] loss: 0.48464425951242446
[Epoch 3: batch 1200] loss: 0.47675515413284303
[Epoch 3: batch 1400] loss: 0.49244742915034295
[Epoch 4: batch 200] l

In [41]:
model.eval()

total_correct = 0
avg_loss = 0.0
total_loss = 0.0
count = 0
for i, data in enumerate(val_loader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    output = model(inputs)
    avg_loss += criterion(output, labels).sum()
    for j in range(output.shape[0]):
        out = output[j]
        probability = torch.softmax(out, -1)[1].cpu().item()
        label = labels[j].cpu().item()
        total_loss += log_loss(probability, label)
        count += 1
    
    pred = output.detach().max(1)[1]
    total_correct += pred.eq(labels.view_as(pred)).sum()

total_loss  = -total_loss/count
print("log loss = ", total_loss)

avg_loss /= len(val_loader.dataset)
avg_loss = avg_loss.detach().cpu().item()
accuracy = float(total_correct) / len(val_loader.dataset)
print(f'Test Avg. Loss: {avg_loss}, Accuracy: {accuracy}')

log loss =  0.36487561005692326
Test Avg. Loss: 0.005715790670365095, Accuracy: 0.8449444444444445


In [0]:
model.eval()

from PIL import Image
dir = "/content/test/test"
files = os.listdir(dir)
results = []
for f in files:
    image = Image.open(dir+"/"+f)
    normalized_image = transform(image)
    normalized_image = normalized_image.unsqueeze(0)
    normalized_image = normalized_image.to(device)
    prediction = model(normalized_image)
    prediction = prediction[0]
    probabilities = torch.softmax(prediction, -1)
    probability = probabilities[1].detach().cpu().item()
    results.append((f, probability))
    

In [0]:
import csv

with open('CrossEntropyLoss_net_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["imagename", "is_fake"])
    for el in results:
        writer.writerow([el[0], el[1]])

In [0]:

save_name = 'NLLLoss_results.csv'
path = F"/content/gdrive/My Drive/{save_name}" 
torch.save("/content/NLLLoss_results.csv", path)